# Friction

In the previous demos, we've looked exclusively at systems that conserve energy.
Many of the problems we're interested in are not purely energy-conserving because the dynamics have dissipative parts as well, such as bottom friction or viscosity.
In this demo, we'll show how to add bottom friction to the shallow water model and how to solve this problem using an *implicit-explicit* timestepping scheme.

All of the schemes we've used so far have been purely explicit -- the system state at the next step is a simple function of its current state.
Implicit schemes instead define the state at the next step as the solution of possibly complex nonlinear system.
These types of schemes are more expensive than their explicit counterparts, but they can give much better results for dynamics with a very rapid relaxation timescale, such as viscous diffusion of momentum.
These types of problems are called *stiff* in the numerical literature.
An implicit-explicit or IMEX method combines both types -- explicit for the non-stiff dynamics, implicit for the stiff dynamics.

In [ ]:
import firedrake
nx, ny = 24, 24
Lx, Ly = 20., 20.
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly, diagonal='crossed')

In [ ]:
degree = 1
Q = firedrake.FunctionSpace(mesh, family='DG', degree=degree)
V = firedrake.VectorFunctionSpace(mesh, family='DG', degree=degree)

Z = Q * V
z_0 = firedrake.Function(Z)

For this demo, we'll use a bed that slopes down from the left to the right-hand side of the domain.
If we used a flat bottom and added friction, the frictional dissipation would slow down the flow and cause the fluid to build up on the inflow boundary.
This kind of flow is closer what we'll encounter when we simulate buoyant meltwater plumes under ice shelves.

In [ ]:
from firedrake import inner, max_value, Constant

x = firedrake.SpatialCoordinate(mesh)

b_0 = Constant(0.0)
δb = Constant(0.2)
b = b_0 - δb * x[0] / Lx

The initial state will be roughly the same as the last demo.

In [ ]:
H = Constant(1.0)
u_in = Constant(2.5)
q_in = H * Constant((u_in, 0.0))

h_0, q_0 = z_0.split()
h_0.project(H)
q_0.project(q_in);

In [ ]:
import numpy as np
from plumes.coefficients import gravity
C = abs(float(u_in)) + np.sqrt(gravity * float(H))
δx = mesh.cell_sizes.dat.data_ro[:].min()
timestep = (δx / 8) / C / (2 * degree + 1)

final_time = 8 * Lx / C
num_steps = int(final_time / timestep)
dt = final_time / num_steps

output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)

Here's where things start to get different.
First, we'll make the shallow water wave equation like we always have.
This is the non-stiff part of the dynamics.

In [ ]:
from plumes import models
g = Constant(gravity)
wave_equation = models.shallow_water.make_equation(
    g, b, h_in=H, q_in=q_in, inflow_ids=(1,), outflow_ids=(2,)
)

Next we'll make the friction equation; this is the stiff part of the dynamics.
Most models assume that bottom friction is proportional to the square magnitude of velocity:

$$\partial_tq + \nabla\cdot F = -gh\nabla b - k|u|u.$$

If you work out the units, the friction coefficient $k$ is dimensionless, which is especially nice.
The code below shows the inner structure of what a function looks like that calculates the weak form of part of the problem.
This function takes in the state variable $z$ contains both thickness and momentum and outputs a `firedrake.Form` object of rank 1.
We can get symbolic representations of the components of the state variable using the function `firedrake.split`.
All of the other equation builders, like the wave equation builder we just called above, do something roughly similar.

In [ ]:
from firedrake import sqrt, dx

ξ = Constant((Lx / 2, Lx / 2))
R = Constant(Lx / 8)
k_0 = Constant(1.0)
k = k_0 * max_value(0, 1 - inner(x - ξ, x - ξ) / R**2)

def friction_equation(z):
    Z = z.function_space()
    ϕ, v = firedrake.TestFunctions(Z)
    h, q = firedrake.split(z)

    u = q / h
    U = sqrt(inner(u, u))
    return -k * U * inner(u, v) * dx

Now we'll create an IMEX integrator.
The IMEX integrator has a slightly different interface compared to the ones we've seen before.
The integration scheme can't magically figure out which part of the dynamics is stiff and which isn't, so we have to pass the two equations separately.

Additionally, we're specifying the parameters to pass to the Firedrake form compiler.
This step isn't strictly necessary, but the form compiler is overly conservative and will use a very high-degree quadrature rule at the expense of speed.

In [ ]:
from plumes import numerics
params = {
    'form_compiler_parameters': {
        'quadrature_degree': 4
    }
}
integrator = numerics.IMEX(
    wave_equation, friction_equation, z_0, dt, **params
)

In every other respect, we use the IMEX integrator the same way.

In [ ]:
import tqdm

hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

And the same old rigmarole to make a movie.

In [ ]:
%%capture
Q0 = firedrake.FunctionSpace(mesh, family='DG', degree=0)
η = firedrake.project(hs[0] + b, Q0)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
colors = firedrake.tripcolor(
    η, num_sample_points=1, vmin=0.5, vmax=1.2, axes=axes
)
fig.colorbar(colors)

def animate(h):
    η.project(h + b)
    colors.set_array(η.dat.data_ro[:])

interval = 1e3 * output_freq * dt
animation = FuncAnimation(fig, animate, frames=hs, interval=interval)

The fluid immediately builds up on the stoss side of the frictional obstacle and drops sharply on the lee side.
A cool effect you can see in the movie is how the collision of the fluid generates waves that propagate back upstream and bounce off the walls before being forced back in the other direction by the prevailing flow.

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

We could use the same mechanism shown above to add other kinds of dynamics to the system.
For example, we could add sources of mass, viscous diffusion of momentum, or the coriolis effect.
Knowing which kinds of dynamics are stiff or not requires understanding the spectrum of the linearization of the resulting operator.